IMPORTANDO BIBLIOTECAS

In [1]:
import os, shutil, sys, warnings
import errno
import pandas as pd
import seaborn as sns

PARAMETRIZAÇÃO DA APLICAÇÃO

In [2]:
basedirwin = 'D:\\Git\\IBMEC\\Projeto-DAeA\\Dataset\\CSVS\\'
warnings.simplefilter(action='ignore', category=FutureWarning)

REMOVE CSVs VELHOS E COPIA OS NOVOS

In [3]:
if os.path.exists(basedirwin):
    shutil.rmtree(basedirwin)
else:
    sys.exit("Erro ao apagar o diretório de CSVs antigos. Não encontrado.")

try:
    shutil.copytree('D:\\Git\\IBMEC\\Projeto-DAeA\\CSVS\\', 'D:\\Git\\IBMEC\\Projeto-DAeA\\Dataset\\CSVS\\', dirs_exist_ok=True)
except OSError as err:
    sys.exit("Erro copiando a pasta de CSVs novos: % s" % err)

VARIÁVEIS DE AMBIENTE

In [4]:
try:
    listadir = os.listdir(basedirwin)
except OSError as err:
    if err.errno == errno.ENOENT:
        sys.exit("Erro listando o diretório de CSVs fonte: % s" % err)

FORMATANDO O ARQUIVO DE VOLUMES

In [5]:
for dir in listadir:
    with open(basedirwin + dir + '\\particoes.csv', 'r+') as file:
        espaco = file.readlines()
        for ln in espaco:
            file.seek(espaco.index(ln))
            dir2 = dir[0:4] + "/" + dir[4:6] + "/" + dir[6:8]
            ln2 = ln[:-1] + "," + dir2 + "\n"
            with open(basedirwin + dir + '\\particoes2.csv', 'a') as fileNovo:
                fileNovo.write(ln2)
            fileNovo.close()
    file.close()
    os.remove(basedirwin + dir + '\\particoes.csv')
    os.rename(basedirwin + dir + '\\particoes2.csv', basedirwin + dir + '\\particoes.csv')

CARREGANDO OS DADOS DE BACKUP NO DATASET

In [6]:
df_bkp = pd.DataFrame()
for dir in listadir:
    df_bkp = df_bkp.append(pd.read_csv(basedirwin + dir + '\\coletas.csv', header=None))
    # df_bkp = df_bkp.concat(pd.read_csv(basedirwin + dir + '\\coletas.csv'), ignore_index=True)
df_bkp.columns = ['nodename', 'datainicio', 'status', 'result', 'datafim', 'server']
df_bkp['datafim'] = df_bkp['datafim'].astype('datetime64[ns]')
# df_bkp.head()

,nodename,datainicio,status,result,datafim,server
0,MG_SERPRO_BHE_MGBHESERVD305_MENSAL,2022/03/26,Completed,4.0,2022-03-26,TSM_OPBHE
1,MG_SERPRO_BHE_MGBHESERVD093_MENSAL,2022/03/26,Completed,8.0,2022-03-26,TSM_OPBHE
2,MG_SERPRO_BHE_MGBHESERVD187_MENSAL,2022/03/26,Completed,0.0,2022-03-26,TSM_OPBHE
3,MG_SERPRO_BHE_MGBHESERVP240_MENSAL,2022/03/26,Completed,0.0,2022-03-27,TSM_OPBHE
4,MG_SERPRO_BHE_SERVICOSRL_MENSAL,2022/03/26,Completed,0.0,2022-03-26,TSM_OPBHE


CARREGANDO OS DADOS DE VOLUME NO DATASET

In [7]:
df_vol = pd.DataFrame()
for dir in listadir:
    df_vol = df_vol.append(pd.read_csv(basedirwin + dir + '\\particoes.csv', header=None))
df_vol.columns = ['particao', 'total', 'usado', 'livre', 'porcentagem', 'server', 'data']
df_vol['data'] = df_vol['data'].astype('datetime64[ns]')
# df_vol.head(100)

,particao,total,usado,livre,porcentagem,server,data
0,/tsm_fpool,3.7T,2.3T,1.4T,64%,TSM_OPBHE,2022-03-28
1,/tsm_archivelog,838G,598G,241G,72%,TSM_OPBHE,2022-03-28
2,/tsm_backup_db,559G,129G,430G,24%,TSM_OPBHE,2022-03-28
3,/tsm_fpool_copia,8.1T,6.8T,1.4T,84%,TSM_OPBHE,2022-03-28
4,/tsm_db,559G,71G,489G,13%,TSM_OPBHE,2022-03-28
...,...,...,...,...,...,...,...
29,/tsm_fpool_copia,4.0T,1.9T,2.1T,47%,TSM_OPFLA,2022-03-29
30,/tsm_fpool,525G,207G,319G,40%,TSM_OPFNS,2022-03-29
31,/tsm_fpool_copia,1.3T,860G,461G,66%,TSM_OPFNS,2022-03-29
32,/tsm_backup_db,200G,52G,149G,26%,TSM_OPFNS,2022-03-29


GERANDO DATASETS DE PARTIÇÕES E PORCENTAGEM DE USO POR REGIONAL

In [9]:
# Gerar lista de regionais (list_reg)
list_reg = df_vol['server'].unique()

# Percorrendo list_reg, gerando datasets por regional
# ver https://python.plainenglish.io/how-to-dynamically-declare-variables-inside-a-loop-in-python-21e6880aaf8a
for reg in list_reg:
    df_vol_mask = df_vol['server'] == reg
    df_vol_reg = df_vol[df_vol_mask]
    print('Dados da regional' + reg + ':\n') 
    print(df_vol_reg.head(10) + '\n')
    # df_vol[['particao', 'porcentagem', 'data']].copy()
# df_vol_reg.head()

Dados da regionalTSM_OPBHE:



TypeError: unsupported operand type(s) for +: 'DatetimeArray' and 'str'

GERANDO GRÁFICOS COM O SEABORN

In [ ]:
sns.barplot(data=df_vol, x='data', y=)